# Model Training
This notebook defines and trains the neural network model for fire risk classification.

In [1]:
import torch
from torch import nn, optim


## Define Neural Network Architecture

In [ ]:
# defining classifier class
class RiskClassifier(nn.Module):
    def __init__(self, input_size, hidden_size=64, output_size=3):
        super(RiskClassifier, self).__init__()
        #series of linear/RELU/softmax/dropout functions
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size//2)
        self.fc3 = nn.Linear(hidden_size//2, output_size)
        self.softmax = nn.Softmax(dim=1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.batchnorm1 = nn.BatchNorm1d(hidden_size)

    def forward(self, x):
        # trialling with different combinations, not finalized yet
        x = self.fc1(x)
        x = self.batchnorm1(x)
        x = self.relu(x)
        #x = self.softmax(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        #x = self.softmax(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x

## Initialize Model, Loss Function, and Optimizer

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

def init_model(X_train_t, y_train_t):
    # initialize model, loss, optimizer
    model = RiskClassifier(X_train_t.shape[1])
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    y_np = y_train_t.cpu().numpy()
    classes = np.unique(y_np)
    weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_np)
    class_weights = torch.tensor(weights, dtype=torch.float32)

    criterion = nn.CrossEntropyLoss(weight=class_weights)

    return model, criterion, optimizer

## Training Loop

In [4]:
def train_model(model, criterion, optimizer, X_train_t, y_train_t, X_val_t, y_val_t, epochs=30):
    # training loop
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train_t)
        loss = criterion(outputs, y_train_t)
        loss.backward()
        optimizer.step()

        # validation
        model.eval()
    return model

In [ ]:
# combining all steps into a single function
def risk_classifier(data_src="final_dataset.csv", epochs=30):
    X_train_t, y_train_t, X_val_t, y_val_t, X_test_t, y_test_t = prepare_data(data_src)
    model, criterion, optimizer = init_model(X_train_t, y_train_t)
    trained_model = train_model(model, criterion, optimizer, X_train_t, y_train_t, X_val_t, y_val_t, epochs)
    return trained_model, X_test_t, y_test_t